# IMPORTS

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import glob
import cv2
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import *

2024-03-31 18:37:38.554942: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-31 18:37:38.554999: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-31 18:37:38.556530: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Loading Image Files from Dataset Directory

In [2]:
# Loading imgs 
imgs=glob.glob('/kaggle/input/english-handwritten-characters-dataset/Img/*')

# Loading csv file
df = pd.read_csv("/kaggle/input/english-handwritten-characters-dataset/english.csv")
df.head(6)

,image,label
0,Img/img001-001.png,0
1,Img/img001-002.png,0
2,Img/img001-003.png,0
3,Img/img001-004.png,0
4,Img/img001-005.png,0
5,Img/img001-006.png,0


# Data Preparation and Preprocessing for Image Classification Model

In [3]:
# Sorting image paths
new_imgs = sorted(imgs)

# Listing labels
labels = df['label'].tolist()

# Reading images and preprocessing
new_data = []
for i in new_imgs:
    # Read image
    img = cv2.imread(i, 1)
    # Resize image
    img = cv2.resize(img, (64, 64))
    # Append to new_data list
    new_data.append(img)
# Convert new_data to numpy array
new_data = np.array(new_data)

# Convert labels to numpy array
labels = np.array(labels)

# Encode labels
Encoder = LabelEncoder()
labels = Encoder.fit_transform(labels)

# Split data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(new_data, labels, test_size=0.2, random_state=42)

# Normalize pixel values to the range [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0


# Bulding model

In [4]:
model = Sequential()
model.add(Conv2D(512,(5,5),activation='relu',input_shape=(64,64,3)))
model.add(MaxPool2D(2,2))
model.add(Conv2D(256,(3,3),activation='relu',input_shape=(64,64,3)))
model.add(MaxPool2D(2,2))
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(1024,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='sigmoid'))
model.add(Dense(62,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(train_images,train_labels,epochs=20,validation_split = 0.2)

Epoch 1/20
 5/69 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.0401 - loss: 4.3152

I0000 00:00:1711910322.830160     311 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1711910322.846889     311 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.0180 - loss: 4.2275

W0000 00:00:1711910329.243048     312 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1711910330.010384     312 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


69/69 ━━━━━━━━━━━━━━━━━━━━ 20s 126ms/step - accuracy: 0.0180 - loss: 4.2270 - val_accuracy: 0.0110 - val_loss: 4.1941
Epoch 2/20
 5/69 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.0000e+00 - loss: 4.1230

W0000 00:00:1711910331.425134     313 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.0157 - loss: 4.1240 - val_accuracy: 0.0769 - val_loss: 3.7523
Epoch 3/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.1111 - loss: 3.4613 - val_accuracy: 0.1905 - val_loss: 2.9614
Epoch 4/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.3184 - loss: 2.5404 - val_accuracy: 0.3663 - val_loss: 2.3918
Epoch 5/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.4441 - loss: 1.8973 - val_accuracy: 0.4615 - val_loss: 1.9160
Epoch 6/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.6011 - loss: 1.4056 - val_accuracy: 0.6044 - val_loss: 1.4670
Epoch 7/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.6944 - loss: 1.0710 - val_accuracy: 0.6227 - val_loss: 1.2783
Epoch 8/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.8023 - loss: 0.7007 - val_accuracy: 0.6667 - val_loss: 1.1737
Epoch 9/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.8560 - loss: 0.5297 - val_accuracy: 0.6685 - val_loss: 1.

# Evaluate model

In [5]:
model.evaluate(test_images,test_labels)

22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 102ms/step - accuracy: 0.6953 - loss: 1.2380


W0000 00:00:1711910381.571255     311 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


[1.081328272819519, 0.7316715717315674]

# Make predictions 

In [6]:
y_pred = model.predict(test_images)
y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate evaluation metrics
accuracy = accuracy_score(test_labels, y_pred_labels)
precision = precision_score(test_labels, y_pred_labels, average='weighted')
recall = recall_score(test_labels, y_pred_labels, average='weighted')
f1 = f1_score(test_labels, y_pred_labels, average='weighted')
confusion_mat = confusion_matrix(test_labels, y_pred_labels)

# Print or use the evaluation metrics as needed
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion_mat)

20/22 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

W0000 00:00:1711910382.153806     313 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step
Accuracy: 0.7316715542521994
Precision: 0.759280337817671
Recall: 0.7316715542521994
F1 Score: 0.7304782438712825
Confusion Matrix:
[[ 3  0  0 ...  0  0  0]
 [ 0  6  0 ...  0  0  0]
 [ 0  0  9 ...  0  0  1]
 ...
 [ 0  0  0 ... 11  0  2]
 [ 0  0  0 ...  0  4  0]
 [ 0  0  0 ...  0  0  9]]


W0000 00:00:1711910382.749404     311 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


# VISUALIZATION

In [16]:
# Calculate precision and recall
precision = precision_score(test_labels, y_pred_labels, average='weighted')
recall = recall_score(test_labels, y_pred_labels, average='weighted')

# Print precision and recall
print("Precision:", precision)
print("Recall:", recall)


Precision: 0.759280337817671
Recall: 0.7316715542521994
